# Hubbard calculations

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

## Specifying the Hubbard model for a four-site chain

If we would like to investigate a four-site chain, we should specify the adjacency matrix and the Hubbard model parameters `t` and `U`. Then, we can make a hopping matrix.

In [2]:
A = gqcpy.AdjacencyMatrix.Linear(4)
print(A.matrix())

AttributeError: module 'gqcpy' has no attribute 'AdjacencyMatrix'

In [3]:
t = 1.0
U = 3.5
H = gqcpy.HoppingMatrix(A, t, U)
print(H.matrix())

[[ 3.5 -1.   0.   0. ]
 [-1.   3.5 -1.   0. ]
 [ 0.  -1.   3.5 -1. ]
 [ 0.   0.  -1.   3.5]]


This hopping matrix can be used to construct the Hubbard model Hamiltonian.

In [4]:
hubbard_hamiltonian = gqcpy.HubbardHamiltonian(H)
print(hubbard_hamiltonian.core().parameters())

[[ 0. -1.  0.  0.]
 [-1.  0. -1.  0.]
 [ 0. -1.  0. -1.]
 [ 0.  0. -1.  0.]]


## Dense Hubbard calculations

The first thing we should take care of is the use of the correct ONV basis. 

In [5]:
K = 4  # number of sites
N_P = 2  # number of electron pairs

onv_basis = gqcpy.SpinResolvedONVBasis(K, N_P, N_P)

In GQCP, the Hubbard model is treated as a model Hamiltonian. What this means is that, behind the scenes, it is not a regular `SQHamiltonian`, but combined with the associated `SpinResolvedONVBasis`, it uses specialized matrix construction and matrix-vector product routines.

Even though it _isn't_ a `SQHamiltonian`, we could say it behaves as such, so doing Hubbard (CI) calculations resembles doing FCI calculations. We'll have to specify:
1. a solver;
1. an associated environment,

and then combine these arguments in the `CI` `QCMethod`.

In [6]:
solver = gqcpy.EigenproblemSolver.Dense()
environment = gqcpy.CIEnvironment.Dense(hubbard_hamiltonian, onv_basis)

energy = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateEnergy()
print(energy)

-2.135871608231944


This is exactly the energy we find from a [reference implementation](https://github.com/wpoely86/Hubbard-GPU) by Ward Poelmans.

## Hubbard calculations with a Davidson algorithm

Doing iterative Hubbard calculations (i.e. with a Davidson algorithm) is also similar to doing FCI calculations. We specify a solver, an environment and combine them into the `CI` `QCMethod`.

In [7]:
x0 = gqcpy.LinearExpansion_SpinResolved.Random(onv_basis).coefficients()

solver = gqcpy.EigenproblemSolver.Davidson()
environment = gqcpy.CIEnvironment.Iterative(hubbard_hamiltonian, onv_basis, x0)

energy = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateEnergy()
print(energy)

-2.1358716082319384
